# Random Forest Test

In [1]:
import pandas as pd
import numpy as np

## Definindos os metodos para o Pipeline

In [2]:
def build_cube(col, row, week):
    X = np.array([
        df.col.values,
        df.row.values,
        df.week_number.values
    ])

    new_x = [[], [], []]
    for i in range(len(X[0])):
        if 0 < X[2][i] < 261:
            new_x[0].append(X[0][i])
            new_x[1].append(X[1][i])
            new_x[2].append(X[2][i])

    X = new_x
    
    dim = [max(X[2]) + 1, max(X[1]) + 1, max(X[0]) + 1]

    cube = []
    for i in range(dim[0]):
        cube.append([])

        for j in range(dim[1]):
            cube[i].append([])

            for k in range(dim[2]):
                cube[i][j].append(0)
                
    for i in range(len(X[0])):
        cube[X[2][i]][X[1][i]][X[0][i]] += 1
    
    return cube

In [3]:
def split_cube(cube):
    new_x = []
    y = []
    for w in range(len(cube)):
        for r in range(len(cube[0])):
            for c in range(len(cube[0][0])):
                new_x.append(np.array([w,r,c]))
                y.append(cube[w][r][c])
    return np.array(new_x), y

def convert_y_to_binary(y):
    y2 = pd.DataFrame(y, columns=['y']) > 0
    return y2.astype(int).values

# Random Forest

In [4]:
from sklearn.model_selection import train_test_split

# Separando Treino e Teste
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import  RandomForestClassifier
from sklearn import metrics

def predict_with_rf(X_train, X_test, y_train, y_test):
    clf = Pipeline([('sc', StandardScaler()), ('rf', RandomForestClassifier())])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
    return y_pred

## Convertendo os resultados da predição e os dados reais para um cubo [Semana, Coluna e Linha]

In [6]:
def result_to_cube(y_test, y_pred):
    real = [[],[],[]]
    pred = [[],[],[]]
    for i in range(len(y_pred)):
        x = X_test[i]
        if y_test[i][0] > 0:
            for l in range(y_test[i][0]):
                real[0].append(x[0])
                real[1].append(x[1])
                real[2].append(x[2])

        if y_pred[i] > 0:
            for l in range(y_pred[i]):
                pred[0].append(x[0])
                pred[1].append(x[1])
                pred[2].append(x[2])
    cube_r = build_cube(real[0], real[1], real[2])
    cube_p = build_cube(pred[0], pred[1], pred[2])
    return cube_r, cube_p

In [7]:
def compara_acertos(X_test, y_test, y_pred):
    positivo = 0
    acerto = 0
    for i in range(len(y_pred)):
        if y_pred[i] == 1 or y_test[i][0] == 1:
            if  y_pred[i] == y_test[i][0]:
                acerto += 1
            positivo += 1
            
            print(X_test[i], y_pred[i], y_test[i][0],  y_pred[i] == y_test[i][0])
    
    return acerto/positivo

## Pipeline com granularidade 10
Celulas com tamanho de 0.1°

In [8]:
df = pd.read_excel('dataframe_6.xlsx')

In [9]:
X, y = split_cube(build_cube(df.col.values, df.row.values, df.week_number.values))
y = convert_y_to_binary(y)

In [10]:
from sklearn.model_selection import train_test_split

# Separando Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [10]:
y_pred = predict_with_rf(X_train, X_test, y_train, y_test)
acerto = compara_acertos(X_test, y_test, y_pred)
print("Taxa de acerto: ", acerto)

/home/andre/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andre/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andre/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/andre/.local/lib/python3.6/site-packages/sklearn/pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/home/andre/.local/lib/python3.6/site-packages/skle

Accuracy: 0.9071979365377898
[208  24  20] 1 1 True
[208  25   7] 0 1 False
[208  25  15] 1 1 True
[208  25  18] 0 1 False
[208  25  20] 1 1 True
[208  26   9] 0 1 False
[208  26  18] 0 1 False
[208  27  18] 0 1 False
[208  28  14] 0 1 False
[209   0   1] 0 1 False
[209   0   4] 1 1 True
[209   0  10] 1 1 True
[209   1   6] 1 1 True
[209   1   7] 1 1 True
[209   1   8] 0 1 False
[209   1   9] 0 1 False
[209   1  10] 1 1 True
[209   1  11] 1 1 True
[209   1  12] 1 1 True
[209   2   1] 1 1 True
[209   2   2] 1 1 True
[209   2   3] 1 0 False
[209   2   4] 1 1 True
[209   2   5] 1 1 True
[209   2   6] 1 1 True
[209   2   7] 0 1 False
[209   2   9] 1 1 True
[209   2  10] 0 1 False
[209   2  11] 0 1 False
[209   2  12] 1 1 True
[209   3   5] 1 1 True
[209   3   6] 1 0 False
[209   3   7] 1 1 True
[209   3  10] 1 1 True
[209   3  12] 1 1 True
[209   3  13] 1 1 True
[209   4   1] 1 1 True
[209   4   4] 1 0 False
[209   4   5] 1 1 True
[209   4   8] 1 0 False
[209   4   9] 1 0 False
[209   4  1

[213   8  10] 1 0 False
[213   8  12] 0 1 False
[213   8  13] 0 1 False
[213   8  14] 1 1 True
[213   8  15] 1 1 True
[213   8  16] 1 1 True
[213   9   6] 1 1 True
[213   9  10] 1 1 True
[213   9  11] 1 0 False
[213   9  12] 0 1 False
[213   9  13] 1 1 True
[213   9  14] 1 0 False
[213   9  15] 1 1 True
[213   9  16] 1 1 True
[213  10  11] 0 1 False
[213  10  12] 1 1 True
[213  10  13] 1 1 True
[213  10  14] 0 1 False
[213  10  15] 0 1 False
[213  10  16] 0 1 False
[213  11   9] 1 1 True
[213  11  13] 1 0 False
[213  11  14] 1 0 False
[213  11  15] 1 1 True
[213  11  17] 1 1 True
[213  12  11] 0 1 False
[213  12  14] 1 1 True
[213  12  15] 0 1 False
[213  12  17] 1 1 True
[213  13  14] 1 0 False
[213  13  15] 1 1 True
[213  13  19] 0 1 False
[213  14   7] 1 1 True
[213  14   9] 1 0 False
[213  14  11] 1 1 True
[213  14  15] 1 1 True
[213  14  16] 1 1 True
[213  15   7] 0 1 False
[213  15  11] 1 1 True
[213  15  12] 0 1 False
[213  15  14] 0 1 False
[213  15  16] 1 0 False
[213  15  17]

[217   2  12] 1 1 True
[217   3   1] 0 1 False
[217   3   5] 1 0 False
[217   3   6] 1 1 True
[217   3   7] 1 0 False
[217   3  10] 1 0 False
[217   3  12] 1 1 True
[217   3  13] 1 1 True
[217   4   1] 1 1 True
[217   4   4] 1 0 False
[217   4   5] 1 1 True
[217   4   7] 0 1 False
[217   4   8] 1 0 False
[217   4   9] 1 0 False
[217   4  13] 1 1 True
[217   4  14] 1 1 True
[217   5   8] 1 0 False
[217   5   9] 1 0 False
[217   5  10] 1 1 True
[217   5  11] 1 1 True
[217   5  14] 1 1 True
[217   6   1] 0 1 False
[217   6   2] 1 1 True
[217   6   6] 1 1 True
[217   6   7] 0 1 False
[217   6  11] 1 1 True
[217   6  13] 1 1 True
[217   6  14] 1 1 True
[217   6  15] 1 1 True
[217   7   2] 1 1 True
[217   7   7] 0 1 False
[217   7  13] 1 1 True
[217   7  14] 1 1 True
[217   7  15] 1 1 True
[217   7  16] 1 0 False
[217   8   5] 0 1 False
[217   8   9] 0 1 False
[217   8  10] 1 1 True
[217   8  14] 1 1 True
[217   8  15] 1 1 True
[217   8  16] 1 1 True
[217   9   6] 1 1 True
[217   9  10] 1 1 

[220  24   8] 1 0 False
[220  24  11] 0 1 False
[220  24  14] 0 1 False
[220  24  15] 1 1 True
[220  24  16] 1 1 True
[220  24  20] 1 1 True
[220  25  15] 1 1 True
[220  25  18] 0 1 False
[220  25  20] 1 1 True
[220  26   9] 0 1 False
[220  26  18] 0 1 False
[220  27  10] 0 1 False
[220  27  18] 0 1 False
[220  28   9] 0 1 False
[220  28  14] 0 1 False
[221   0   4] 1 1 True
[221   0  10] 1 1 True
[221   1   6] 1 1 True
[221   1   7] 1 1 True
[221   1   8] 0 1 False
[221   1   9] 0 1 False
[221   1  10] 1 1 True
[221   1  11] 1 1 True
[221   1  12] 1 1 True
[221   2   1] 1 1 True
[221   2   2] 1 1 True
[221   2   3] 1 1 True
[221   2   4] 1 1 True
[221   2   5] 1 0 False
[221   2   6] 1 0 False
[221   2   9] 1 1 True
[221   2  11] 0 1 False
[221   2  12] 1 1 True
[221   3   1] 0 1 False
[221   3   5] 1 1 True
[221   3   6] 1 1 True
[221   3   7] 1 1 True
[221   3  10] 1 1 True
[221   3  11] 0 1 False
[221   3  12] 1 1 True
[221   3  13] 1 1 True
[221   4   1] 1 0 False
[221   4   4] 1 

...

[243   4   3] 0 1 False
[243   4   4] 1 0 False
[243   4   5] 1 1 True
[243   4   8] 1 0 False
[243   4   9] 1 0 False
[243   4  13] 1 1 True
[243   4  14] 1 1 True
[243   5   3] 0 1 False
[243   5   8] 1 0 False
[243   5   9] 1 0 False
[243   5  10] 1 0 False
[243   5  11] 1 1 True
[243   5  13] 0 1 False
[243   5  14] 1 1 True
[243   6   2] 1 1 True
[243   6   5] 0 1 False
[243   6   6] 1 1 True
[243   6   7] 0 1 False
[243   6   9] 0 1 False
[243   6  11] 1 1 True
[243   6  12] 0 1 False
[243   6  13] 1 1 True
[243   6  14] 1 1 True
[243   6  15] 1 1 True
[243   7   1] 0 1 False
[243   7   2] 1 1 True
[243   7  13] 1 1 True
[243   7  14] 1 1 True
[243   7  15] 1 1 True
[243   7  16] 1 1 True
[243   8   7] 0 1 False
[243   8   9] 0 1 False
[243   8  10] 1 1 True
[243   8  13] 0 1 False
[243   8  14] 1 1 True
[243   8  15] 1 1 True
[243   8  16] 1 1 True
[243   9   6] 1 1 True
[243   9  10] 1 1 True
[243   9  11] 1 0 False
[243   9  12] 0 1 False
[243   9  13] 1 1 True
[243   9  14] 1

[246  21   7] 1 1 True
[246  21  13] 1 0 False
[246  21  14] 0 1 False
[246  22   6] 0 1 False
[246  22   8] 1 0 False
[246  22   9] 1 1 True
[246  22  13] 1 1 True
[246  22  14] 1 1 True
[246  22  15] 1 1 True
[246  22  19] 1 1 True
[246  22  20] 1 1 True
[246  23   8] 1 0 False
[246  23  11] 0 1 False
[246  23  12] 0 1 False
[246  23  13] 1 1 True
[246  23  14] 1 1 True
[246  23  19] 1 1 True
[246  24   8] 1 0 False
[246  24  14] 0 1 False
[246  24  15] 1 1 True
[246  24  16] 1 1 True
[246  24  20] 1 1 True
[246  25  15] 1 1 True
[246  25  18] 0 1 False
[246  25  20] 1 0 False
[246  26   9] 0 1 False
[246  26  18] 0 1 False
[246  27  18] 0 1 False
[246  28  12] 0 1 False
[246  28  14] 0 1 False
[246  28  15] 0 1 False
[246  29  12] 0 1 False
[247   0   4] 1 1 True
[247   0   6] 0 1 False
[247   0   7] 0 1 False
[247   0   8] 0 1 False
[247   0   9] 0 1 False
[247   0  10] 1 0 False
[247   1   6] 1 1 True
[247   1   7] 1 1 True
[247   1   8] 0 1 False
[247   1   9] 0 1 False
[247   1 

[250  11  13] 1 0 False
[250  11  14] 1 1 True
[250  11  15] 1 1 True
[250  11  17] 1 1 True
[250  12  11] 0 1 False
[250  12  14] 1 1 True
[250  12  15] 0 1 False
[250  12  17] 1 0 False
[250  13  14] 1 1 True
[250  13  15] 1 1 True
[250  14   7] 1 1 True
[250  14   9] 1 0 False
[250  14  10] 0 1 False
[250  14  11] 1 1 True
[250  14  15] 1 1 True
[250  14  16] 1 1 True
[250  15  11] 1 1 True
[250  15  12] 0 1 False
[250  15  16] 1 0 False
[250  15  17] 1 1 True
[250  16  12] 1 0 False
[250  16  17] 1 1 True
[250  17  10] 0 1 False
[250  17  12] 0 1 False
[250  17  13] 0 1 False
[250  17  14] 1 1 True
[250  17  17] 1 0 False
[250  18  11] 1 1 True
[250  18  16] 1 1 True
[250  18  17] 1 1 True
[250  19  12] 1 1 True
[250  20  10] 1 1 True
[250  20  14] 0 1 False
[250  20  17] 1 1 True
[250  20  20] 1 1 True
[250  21   6] 1 1 True
[250  21   7] 1 1 True
[250  21  10] 0 1 False
[250  21  13] 1 0 False
[250  22   8] 1 0 False
[250  22   9] 1 1 True
[250  22  13] 1 1 True
[250  22  14] 1 1

[254   2   3] 1 0 False
[254   2   4] 1 1 True
[254   2   5] 1 1 True
[254   2   6] 1 1 True
[254   2   9] 1 1 True
[254   2  12] 1 1 True
[254   3   5] 1 1 True
[254   3   6] 1 1 True
[254   3   7] 1 1 True
[254   3  10] 1 1 True
[254   3  12] 1 1 True
[254   3  13] 1 1 True
[254   4   1] 1 0 False
[254   4   4] 1 0 False
[254   4   5] 1 1 True
[254   4   8] 1 0 False
[254   4   9] 1 0 False
[254   4  12] 0 1 False
[254   4  13] 1 1 True
[254   4  14] 1 1 True
[254   5   3] 0 1 False
[254   5   8] 1 0 False
[254   5   9] 1 0 False
[254   5  10] 1 0 False
[254   5  11] 1 1 True
[254   5  13] 0 1 False
[254   5  14] 1 1 True
[254   6   2] 1 1 True
[254   6   4] 0 1 False
[254   6   6] 1 1 True
[254   6   7] 0 1 False
[254   6   9] 0 1 False
[254   6  11] 1 1 True
[254   6  13] 1 1 True
[254   6  14] 1 1 True
[254   6  15] 1 1 True
[254   7   2] 1 1 True
[254   7  13] 1 1 True
[254   7  14] 1 1 True
[254   7  15] 1 1 True
[254   7  16] 1 0 False
[254   8   5] 0 1 False
[254   8  10] 1 0 

[257  22  13] 1 1 True
[257  22  14] 1 1 True
[257  22  15] 1 1 True
[257  22  19] 1 1 True
[257  22  20] 1 1 True
[257  23   8] 1 0 False
[257  23  13] 1 1 True
[257  23  14] 1 1 True
[257  23  19] 1 1 True
[257  24   8] 1 0 False
[257  24  15] 1 1 True
[257  24  16] 1 0 False
[257  24  20] 1 1 True
[257  25  15] 1 1 True
[257  25  18] 0 1 False
[257  25  20] 1 1 True
[257  26   9] 0 1 False
[257  26  18] 0 1 False
[257  28  12] 0 1 False
[257  28  14] 0 1 False
[257  28  15] 0 1 False
[257  29  12] 0 1 False
[257  29  22] 0 1 False
[257  30  12] 0 1 False
[258   0   4] 1 1 True
[258   0  10] 1 1 True
[258   1   6] 1 1 True
[258   1   7] 1 1 True
[258   1   8] 0 1 False
[258   1   9] 0 1 False
[258   1  10] 1 1 True
[258   1  11] 1 1 True
[258   1  12] 1 1 True
[258   2   1] 1 1 True
[258   2   2] 1 1 True
[258   2   3] 1 0 False
[258   2   4] 1 1 True
[258   2   5] 1 1 True
[258   2   6] 1 1 True
[258   2   9] 1 1 True
[258   2  12] 1 1 True
[258   3   0] 0 1 False
[258   3   5] 1 1 

## Pipeline com granularidade 100
Celulas com tamanho de 0.01°

In [11]:
df = pd.read_excel('dataframe_7.xlsx')

In [12]:
X, y = split_cube(build_cube(df.col.values, df.row.values, df.week_number.values))
y = convert_y_to_binary(y)

In [13]:
from sklearn.model_selection import train_test_split

# Separando Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [13]:

y_pred = predict_with_rf(X_train, X_test, y_train, y_test)
acerto = compara_acertos(X_test, y_test, y_pred)
print("Taxa de acerto: ", acerto)

/home/andre/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andre/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andre/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/andre/.local/lib/python3.6/site-packages/sklearn/pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/home/andre/.local/lib/python3.6/site-packages/skle

Accuracy: 0.995159941568534
[208 245 150] 0 1 False
[208 246 150] 0 1 False
[208 250 187] 1 0 False
[208 252  78] 0 1 False
[208 254 185] 0 1 False
[208 256 187] 0 1 False
[208 258 158] 0 1 False
[208 258 209] 0 1 False
[208 262  96] 0 1 False
[208 262  97] 1 1 True
[208 263  96] 1 0 False
[208 269 184] 1 1 True
[208 270 184] 1 1 True
[208 271 184] 0 1 False
[208 286 143] 0 1 False
[208 287 143] 0 1 False
[208 290 127] 1 0 False
[209   4  46] 0 1 False
[209   5  18] 0 1 False
[209   5  45] 1 0 False
[209   6  48] 1 0 False
[209   9 101] 1 1 True
[209  10 101] 0 1 False
[209  10 102] 1 1 True
[209  10 103] 1 0 False
[209  11  69] 1 0 False
[209  11 102] 1 0 False
[209  11 103] 1 1 True
[209  12 103] 0 1 False
[209  13  67] 0 1 False
[209  13  68] 1 0 False
[209  13  69] 0 1 False
[209  13  86] 0 1 False
[209  14  64] 0 1 False
[209  14  65] 1 0 False
[209  14  66] 0 1 False
[209  14  67] 0 1 False
[209  14  68] 1 1 True
[209  14  69] 1 1 True
[209  14  70] 1 1 True
[209  14 104] 1 0 Fal

[209 177  95] 0 1 False
[209 180 169] 0 1 False
[209 180 170] 1 1 True
[209 180 209] 1 0 False
[209 181 169] 0 1 False
[209 183 161] 1 0 False
[209 184 172] 0 1 False
[209 185 117] 1 1 True
[209 194  78] 0 1 False
[209 196 125] 1 0 False
[209 197 126] 1 0 False
[209 198 126] 1 1 True
[209 200 140] 1 0 False
[209 200 141] 0 1 False
[209 200 179] 0 1 False
[209 205 203] 1 0 False
[209 207 172] 0 1 False
[209 208 121] 0 1 False
[209 208 172] 1 0 False
[209 209 172] 1 1 True
[209 211  78] 1 0 False
[209 211 184] 0 1 False
[209 213  67] 1 1 True
[209 214 131] 1 0 False
[209 222  89] 0 1 False
[209 222 204] 1 1 True
[209 223  90] 0 1 False
[209 223  91] 0 1 False
[209 224  89] 1 0 False
[209 224  90] 1 1 True
[209 224  91] 0 1 False
[209 224 204] 1 1 True
[209 224 205] 1 1 True
[209 225 194] 0 1 False
[209 225 203] 1 1 True
[209 225 204] 1 1 True
[209 225 205] 0 1 False
[209 226  90] 0 1 False
[209 226 157] 1 0 False
[209 226 158] 0 1 False
[209 226 203] 1 0 False
[209 226 204] 1 1 True
[209

[210  84 155] 1 1 True
[210  84 156] 1 0 False
[210  84 160] 1 1 True
[210  84 161] 1 1 True
[210  85  50] 0 1 False
[210  85 150] 1 1 True
[210  85 151] 0 1 False
[210  85 152] 1 1 True
[210  85 153] 1 1 True
[210  85 154] 1 1 True
[210  85 155] 1 1 True
[210  85 156] 1 0 False
[210  85 161] 1 1 True
[210  86 148] 0 1 False
[210  86 149] 1 1 True
[210  86 150] 1 1 True
[210  86 151] 1 0 False
[210  86 154] 1 1 True
[210  86 155] 0 1 False
[210  86 156] 1 0 False
[210  86 157] 0 1 False
[210  86 159] 1 0 False
[210  86 160] 1 1 True
[210  86 161] 1 1 True
[210  87 148] 1 0 False
[210  87 149] 1 1 True
[210  87 150] 1 1 True
[210  87 159] 1 1 True
[210  87 160] 1 1 True
[210  87 161] 1 1 True
[210  88 130] 0 1 False
[210  88 147] 1 0 False
[210  88 148] 1 0 False
[210  88 150] 1 0 False
[210  88 161] 0 1 False
[210  88 162] 1 1 True
[210  89 161] 1 0 False
[210  90 127] 0 1 False
[210  91  67] 0 1 False
[210  91 119] 1 0 False
[210  92  67] 1 0 False
[210  93 160] 0 1 False
[210  94 158

[211  67 143] 1 1 True
[211  67 144] 1 1 True
[211  67 145] 1 1 True
[211  67 146] 1 1 True
[211  67 147] 1 1 True
[211  67 149] 1 1 True
[211  67 150] 0 1 False
[211  67 151] 0 1 False
[211  67 152] 1 1 True
[211  68  99] 0 1 False
[211  68 136] 0 1 False
[211  68 140] 0 1 False
[211  68 141] 1 1 True
[211  68 142] 1 0 False
[211  68 143] 0 1 False
[211  68 144] 1 1 True
[211  68 145] 1 1 True
[211  68 146] 1 1 True
[211  68 147] 1 1 True
[211  68 148] 1 1 True
[211  68 149] 1 1 True
[211  68 150] 1 1 True
[211  68 151] 0 1 False
[211  68 152] 1 0 False
[211  69 140] 1 1 True
[211  69 141] 1 1 True
[211  69 142] 0 1 False
[211  69 143] 1 1 True
[211  69 144] 1 1 True
[211  69 147] 1 1 True
[211  69 148] 1 1 True
[211  69 149] 1 1 True
[211  69 150] 1 1 True
[211  69 151] 1 1 True
[211  70 139] 1 1 True
[211  70 140] 1 0 False
[211  70 141] 0 1 False
[211  70 143] 0 1 False
[211  70 145] 0 1 False
[211  70 147] 1 1 True
[211  70 148] 1 1 True
[211  70 149] 1 1 True
[211  70 150] 1 1 Tr

...

[258  45  80] 1 0 False
[258  46  13] 0 1 False
[258  46  14] 1 1 True
[258  46 124] 0 1 False
[258  46 141] 1 0 False
[258  47 132] 1 0 False
[258  47 142] 0 1 False
[258  48 143] 0 1 False
[258  48 144] 1 0 False
[258  49 143] 0 1 False
[258  50 102] 0 1 False
[258  51 144] 1 0 False
[258  53  88] 1 0 False
[258  53 131] 0 1 False
[258  55 105] 1 1 True
[258  55 143] 1 0 False
[258  55 144] 1 0 False
[258  55 145] 1 1 True
[258  55 146] 1 1 True
[258  56  96] 1 1 True
[258  56 143] 0 1 False
[258  56 144] 1 1 True
[258  56 145] 1 0 False
[258  56 146] 1 1 True
[258  56 147] 0 1 False
[258  57 146] 1 1 True
[258  57 147] 1 1 True
[258  58 112] 0 1 False
[258  59 112] 0 1 False
[258  59 113] 1 0 False
[258  60 130] 1 0 False
[258  60 147] 0 1 False
[258  60 148] 1 0 False
[258  61  76] 1 0 False
[258  61  77] 0 1 False
[258  61 118] 1 0 False
[258  61 137] 1 0 False
[258  61 139] 0 1 False
[258  61 142] 1 1 True
[258  61 146] 1 1 True
[258  61 147] 1 1 True
[258  61 148] 1 1 True
[258 

[258 224  89] 1 0 False
[258 224  90] 1 1 True
[258 224 204] 1 1 True
[258 224 205] 1 0 False
[258 225 193] 0 1 False
[258 225 203] 1 1 True
[258 225 204] 1 1 True
[258 226 143] 0 1 False
[258 226 157] 1 1 True
[258 226 194] 0 1 False
[258 226 203] 1 1 True
[258 226 204] 1 0 False
[258 226 205] 1 0 False
[258 227 137] 0 1 False
[258 227 190] 0 1 False
[258 227 191] 0 1 False
[258 227 193] 0 1 False
[258 227 194] 1 1 True
[258 227 195] 1 0 False
[258 227 196] 0 1 False
[258 227 197] 0 1 False
[258 227 202] 0 1 False
[258 228 139] 0 1 False
[258 228 140] 1 1 True
[258 228 190] 1 0 False
[258 228 191] 0 1 False
[258 228 192] 1 1 True
[258 228 193] 1 1 True
[258 228 194] 1 1 True
[258 228 195] 1 1 True
[258 229 139] 1 1 True
[258 229 140] 1 1 True
[258 230 139] 1 1 True
[258 230 140] 1 1 True
[258 231  94] 0 1 False
[258 231 141] 1 0 False
[258 231 193] 1 1 True
[258 232 138] 1 0 False
[258 233 100] 0 1 False
[258 233 144] 1 0 False
[258 236  85] 1 0 False
[258 238 200] 0 1 False
[258 241 

[259  84 153] 1 0 False
[259  84 154] 1 1 True
[259  84 155] 1 1 True
[259  84 156] 1 0 False
[259  84 157] 0 1 False
[259  84 160] 1 0 False
[259  84 161] 1 1 True
[259  85 150] 1 1 True
[259  85 152] 1 0 False
[259  85 153] 1 0 False
[259  85 154] 1 1 True
[259  85 155] 1 1 True
[259  85 156] 1 0 False
[259  85 161] 1 1 True
[259  86 148] 0 1 False
[259  86 149] 1 1 True
[259  86 150] 1 1 True
[259  86 151] 1 1 True
[259  86 154] 1 1 True
[259  86 155] 0 1 False
[259  86 156] 1 0 False
[259  86 157] 0 1 False
[259  86 159] 1 1 True
[259  86 160] 1 1 True
[259  86 161] 1 1 True
[259  87 148] 1 1 True
[259  87 149] 1 1 True
[259  87 150] 1 0 False
[259  87 159] 1 1 True
[259  87 160] 1 1 True
[259  87 161] 1 1 True
[259  87 162] 0 1 False
[259  88 147] 1 1 True
[259  88 148] 1 1 True
[259  88 149] 0 1 False
[259  88 150] 1 1 True
[259  88 151] 0 1 False
[259  88 162] 1 1 True
[259  89 127] 0 1 False
[259  89 161] 1 0 False
[259  89 162] 0 1 False
[259  90 146] 0 1 False
[259  91  67] 0

[260  65 143] 0 1 False
[260  65 144] 1 1 True
[260  65 145] 1 1 True
[260  65 146] 1 1 True
[260  65 147] 1 1 True
[260  65 148] 1 1 True
[260  65 149] 1 1 True
[260  65 150] 1 1 True
[260  65 151] 1 1 True
[260  66  67] 1 0 False
[260  66 137] 1 1 True
[260  66 138] 1 1 True
[260  66 139] 1 1 True
[260  66 140] 1 1 True
[260  66 141] 1 1 True
[260  66 142] 1 1 True
[260  66 143] 1 1 True
[260  66 144] 1 1 True
[260  66 145] 1 0 False
[260  66 146] 1 1 True
[260  66 147] 1 1 True
[260  66 148] 1 1 True
[260  66 149] 1 1 True
[260  66 150] 1 1 True
[260  66 151] 1 1 True
[260  66 152] 1 1 True
[260  67  66] 0 1 False
[260  67 136] 0 1 False
[260  67 138] 1 0 False
[260  67 139] 0 1 False
[260  67 140] 1 1 True
[260  67 141] 1 1 True
[260  67 142] 1 1 True
[260  67 143] 1 1 True
[260  67 144] 1 1 True
[260  67 145] 1 1 True
[260  67 146] 1 1 True
[260  67 147] 1 0 False
[260  67 149] 1 1 True
[260  67 152] 1 1 True
[260  68  99] 0 1 False
[260  68 121] 0 1 False
[260  68 136] 0 1 False


[260 286 144] 0 1 False
[260 287 125] 0 1 False
[260 287 126] 0 1 False
[260 287 143] 0 1 False
[260 290 127] 1 0 False
[260 306 125] 0 1 False
Taxa de acerto:  0.4713565123401189


## Pipeline com granularidade 1000
Celulas com tamanho de 0.001°

In [14]:
# df = pd.read_excel('dataframe_8.xlsx')

In [15]:
'''
X, y = split_cube(build_cube(df.col.values, df.row.values, df.week_number.values))
y = convert_y_to_binary(y)
'''

'\nX, y = split_cube(build_cube(df.col.values, df.row.values, df.week_number.values))\ny = convert_y_to_binary(y)\n'

In [16]:
'''
from sklearn.model_selection import train_test_split

# Separando Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

y_pred = predict_with_rf(X_train, X_test, y_train, y_test)
acerto = compara_acertos(X_test, y_test, y_pred)
print("Taxa de acerto: ", acerto)
'''

'\nfrom sklearn.model_selection import train_test_split\n\n# Separando Treino e Teste\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)\n\ny_pred = predict_with_rf(X_train, X_test, y_train, y_test)\nacerto = compara_acertos(X_test, y_test, y_pred)\nprint("Taxa de acerto: ", acerto)\n'

In [11]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
l0 = tf.keras.layers.Dense(units=1, input_shape=[3]) 
model = tf.keras.Sequential([l0])
model.compile(optimizer=tf.keras.optimizers.SGD())
history = model.fit(X_train, y_train, epochs=10, verbose=False)

In [15]:
result = model.predict(X_test)

In [16]:
result

array([[-2.6353285],
       [-2.6438603],
       [-2.6523921],
       ...,
       [-3.291924 ],
       [-3.3004556],
       [-3.3089874]], dtype=float32)

In [17]:
y_train

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])